In [ ]:
import os
from datetime import datetime
import pandas as pd
import requests
import warnings


In [1]:
def get_allstock_info():
    # 獲取所有股票的信息，包括上市和上櫃股票
    url = "https://isin.twse.com.tw/isin/class_main.jsp?owncode=&stockname=&isincode=&market=1&issuetype=1&industry_code=&Page=1&chklike=Y"
    response = requests.get(url)
    listed = pd.read_html(response.text)[0]
    listed.columns = listed.iloc[0, :]
    listed = listed[["有價證券代號", "有價證券名稱", "市場別", "產業別", "公開發行/上市(櫃)/發行日"]]
    listed = listed.iloc[1:]

    # 上櫃股票
    urlTWO = "https://isin.twse.com.tw/isin/class_main.jsp?owncode=&stockname=&isincode=&market=2&issuetype=&industry_code=&Page=1&chklike=Y"
    response = requests.get(urlTWO)
    listedTWO = pd.read_html(response.text)[0]
    listedTWO.columns = listedTWO.iloc[0, :]
    listedTWO = listedTWO.loc[listedTWO['有價證券別'] == '股票']
    listedTWO = listedTWO[["有價證券代號", "有價證券名稱", "市場別", "產業別", "公開發行/上市(櫃)/發行日"]]

    # 上市和上櫃股票代號處理
    stock_num = pd.concat([listed["有價證券代號"].apply(lambda x: str(x) + ".TW"),
                           listedTWO["有價證券代號"].apply(lambda x: str(x) + ".TWO")], ignore_index=True)
    stock_num.loc[len(stock_num) + 1] = '0050.TW'
    stock_num.loc[len(stock_num) + 1] = '^TWII'
    stock_num = stock_num.sort_values(ascending=False).reset_index(drop=True)

    # 合併所有股票信息
    allstock_info = pd.concat([listed, listedTWO], ignore_index=True)
    allstock_info.columns = ["ID", "有價證券名稱", "市場別", "產業別", "公開發行/上市(櫃)/發行日"]
    allstock_info.set_index('ID', inplace=True)
    return stock_num, allstock_info

In [4]:
symbols, allstock_info = get_allstock_info()

In [5]:
import pandas as pd
import os
base_path = 'C:/Users/User/Desktop/stock/data_test'
save_path = 'C:/Users/User/Desktop/stock/summary'
all_symbols_data = []
for symbol in symbols:
    if symbol == "^TWII":
        print('skip', symbol)
        continue
    file_path = os.path.join(base_path, symbol + '.csv')
    data = pd.read_csv(file_path, parse_dates=True, header=0)
    all_symbols_data.append(data)



skip ^TWII


In [9]:
all_data = pd.concat(all_symbols_data, axis=0)
del all_symbols_data

In [10]:
all_data['Date'] = all_data['Date'].apply(lambda x: datetime.strftime(x, '%Y-%m-%d') if type(x) == datetime else x.split(' ')[0])
all_data.set_index('Date', inplace=True)
all_data_group_by_date = all_data.groupby(level='Date', axis=0)

In [11]:
date_index = all_data_group_by_date.count().index

In [13]:
del allstock_info

In [ ]:
print(all_data.columns)
for col in all_data.columns:
    print(col)

In [ ]:
# delet all_data which ID == '^TWII'
all_data2 = all_data
for i in set(all_data2.loc[:, 'ID']):
    if i == '^TWII':
        print(i)
#all_data2 = all_data2.drop(all_data2[all_data2['ID']=='^TWII'], axis=0)

In [ ]:
for i in set(all_data2.loc[:, 'ID']):
    if i == '^TWII':
        print(i)

In [15]:
for date in date_index[::-1][:2000]:
    date_data = all_data_group_by_date.get_group(date)
    # set index to 'ID'
    date_data.set_index('ID', inplace=True)
    date_data.to_excel(os.path.join(save_path, f"daily_stock_summary_{date}_with_rs_rate_and_maxmin" + '.xlsx'), index=True)